In [210]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics  import confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import numpy as np
import sklearn.linear_model

path = "/Users/noa/Desktop/Business Analytics/"
Stroke = pd.read_csv(path +"healthcare-dataset-stroke-data.csv")

In [211]:
Stroke = Stroke.drop("id", axis = 1)
Stroke = pd.get_dummies(Stroke, drop_first = True)

In [212]:
Stroke_columns = ['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi', 'gender_Male', 'gender_Other', 'ever_married_Yes', 'work_type_Never_worked', 'work_type_Private','work_type_Self-employed', 'work_type_children' ,'Residence_type_Urban','smoking_status_formerly smoked','smoking_status_never smoked','smoking_status_smokes']
len(Stroke_columns)

16

In [213]:
y = Stroke["stroke"]
X = Stroke.drop("stroke", axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1)

In [214]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
imputer.fit(X_train)
X_train_imputed = imputer.transform(X_train)
X_train_imputed = pd.DataFrame(X_train_imputed, columns=Stroke_columns, index = X_train.index)

In [215]:
X_test_imputed = imputer.transform(X_test)
X_test_imputed = pd.DataFrame(X_test_imputed, columns = Stroke_columns, index=X_test.index)
X_test_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1533 entries, 4673 to 3810
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   age                             1533 non-null   float64
 1   hypertension                    1533 non-null   float64
 2   heart_disease                   1533 non-null   float64
 3   avg_glucose_level               1533 non-null   float64
 4   bmi                             1533 non-null   float64
 5   gender_Male                     1533 non-null   float64
 6   gender_Other                    1533 non-null   float64
 7   ever_married_Yes                1533 non-null   float64
 8   work_type_Never_worked          1533 non-null   float64
 9   work_type_Private               1533 non-null   float64
 10  work_type_Self-employed         1533 non-null   float64
 11  work_type_children              1533 non-null   float64
 12  Residence_type_Urban           

In [216]:
X_train_imputed = sm.add_constant(X_train_imputed)
X_train_imputed

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


,const,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
4313,1.0,57.00,0.0,0.0,134.76,29.1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
376,1.0,0.88,0.0,0.0,88.11,15.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4913,1.0,20.00,0.0,0.0,84.49,20.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1791,1.0,13.00,0.0,0.0,137.45,18.2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2166,1.0,28.00,0.0,0.0,169.49,27.2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,1.0,46.00,0.0,0.0,137.77,29.3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2763,1.0,45.00,0.0,0.0,63.73,32.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
905,1.0,31.00,0.0,0.0,76.26,35.6,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3980,1.0,45.00,0.0,0.0,218.10,55.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [217]:
model = sm.OLS(y_train,X_train_imputed)
results = model.fit()
results.params

const                            -9.096046e-02
age                               3.191230e-03
hypertension                      3.463665e-02
heart_disease                     4.549948e-02
avg_glucose_level                 1.738127e-04
bmi                              -4.243912e-04
gender_Male                      -1.192617e-02
gender_Other                     -1.405221e-17
ever_married_Yes                 -4.423265e-02
work_type_Never_worked            3.715676e-02
work_type_Private                 2.276872e-02
work_type_Self-employed          -4.656767e-03
work_type_children                6.813127e-02
Residence_type_Urban              8.181344e-03
smoking_status_formerly smoked   -2.105461e-03
smoking_status_never smoked      -6.538069e-03
smoking_status_smokes            -9.543306e-03
dtype: float64

In [218]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 stroke   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     20.23
Date:                Sat, 07 May 2022   Prob (F-statistic):           2.86e-53
Time:                        13:55:15   Log-Likelihood:                 646.94
No. Observations:                3577   AIC:                            -1262.
Df Residuals:                    3561   BIC:                            -1163.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

In [219]:
standardized_effects = X_train.std() * results.params 
np.exp(standardized_effects)

Residence_type_Urban              1.004100
age                               1.074404
avg_glucose_level                 1.007944
bmi                               0.996695
const                                  NaN
ever_married_Yes                  0.979250
gender_Male                       0.994129
gender_Other                      1.000000
heart_disease                     1.010309
hypertension                      1.010372
smoking_status_formerly smoked    0.999200
smoking_status_never smoked       0.996851
smoking_status_smokes             0.996568
work_type_Never_worked            1.002559
work_type_Private                 1.011329
work_type_Self-employed           0.998290
work_type_children                1.023435
dtype: float64

In [221]:
y_test_pred = results.predict(sm.add_constant(X_test_imputed))
y_test_pred.head(10)

4673    0.000583
3232    0.143401
3694    0.009830
1070    0.145964
4163    0.058505
795    -0.029072
440     0.016977
870     0.035290
1102    0.016151
836     0.109406
dtype: float64

In [222]:
y_test_predclass = (y_test_pred > 0.1).astype(int)
y_test_predclass.head(10)

4673    0
3232    1
3694    0
1070    1
4163    0
795     0
440     0
870     0
1102    0
836     1
dtype: int64

In [223]:
confusion_matrix(y_test, y_test_predclass)

array([[1186,  264],
       [  25,   58]])

In [226]:
""" 
True Negatives = 1186
False Positives = 264
False Negatives = 25
True Positives = 58
"""

' \nTrue Negatives = 1186\nFalse Positives = 264\nFalse Negatives = 25\nTrue Positives = 58\n'

In [227]:
print(classification_report(y_test, y_test_predclass))

              precision    recall  f1-score   support

           0       0.98      0.82      0.89      1450
           1       0.18      0.70      0.29        83

    accuracy                           0.81      1533
   macro avg       0.58      0.76      0.59      1533
weighted avg       0.94      0.81      0.86      1533



In [ ]:
model = sm.GLM(y_train,X_train_imputed)
results = model.fit()
results.params

In [ ]:
print(results.summary())

In [ ]:
standardized_effects = X_train.std() * results.params 
# np.exp(standardized_effects)

In [ ]:
y_test_pred = results.predict(sm.add_constant(X_test_imputed))
# y_test_pred.head(10)

In [ ]:
y_test_predclass = (y_test_pred > 0.1).astype(int)
y_test_predclass.head(10)

In [ ]:
confusion_matrix(y_test, y_test_predclass)

In [ ]:
print(classification_report(y_test, y_test_predclass))